In [5]:
%pwd

'/home/arno/Projects/Pint3D/print_ngp/util/sceneWistProcess'

In [2]:
import numpy as np
from tqdm import tqdm


棕榆树

In [18]:
DataPath="/home/arno/Projects/Pint3D/print_data/fern/volume/ngp_471/array/allData.npy"
data=np.load(DataPath)
data.shape 

# data=np.flip(data, axis=0)

(2857, 383, 408, 4)

In [23]:
data.shape 
# -1.9~0.55对应2857长度
#   玻璃大概在z=-0.27 这里

(2857, 383, 408, 4)

In [19]:
volumeBeyondTheGlass = data[:2857-957, :, :, :].astype(np.float32)

In [20]:
# 沿着第一个axis，每次迭代切出四个切片，叠加这四个切片
statckHeight=4
new_depth = volumeBeyondTheGlass.shape[0] // statckHeight  # 每次迭代切出四个切片
small = np.zeros((new_depth, volumeBeyondTheGlass.shape[1], volumeBeyondTheGlass.shape[2], volumeBeyondTheGlass.shape[3]))


for i in tqdm(range(0, volumeBeyondTheGlass.shape[0], statckHeight)):
    # 获取当前四个切片
    slices = volumeBeyondTheGlass[i:i+statckHeight :, :, :]
    # 分离RGB和D通道
    rgb_slices = slices[..., :3]
    d_slices = slices[..., 3]
    
    # 计算D通道的和
    d_sum = np.sum(d_slices, axis=0)
    # 避免除以零的情况
    d_sum[d_sum == 0] = 1e-8
    weight = np.zeros_like(d_slices)
    for j in range(d_slices.shape[0]):
        weight[j] = d_slices[j] / d_sum

    
        # 计算RGB通道的加权平均
        weighted_rgb = rgb_slices[j] * weight[j][..., np.newaxis]
        
        # 合并RGB和D通道
        small[i//statckHeight, :, :, :3] += weighted_rgb
        
    small[i//statckHeight, :, :, 3] = d_sum


100%|██████████| 475/475 [00:06<00:00, 73.54it/s]


In [21]:
volume=data[2857-957:, :, :, :]
# volume=np.concatenate([volume,small],axis=0)
volume=np.concatenate([small, volume],axis=0)
# volume=np.flip(volume, axis=0)
# small=small[:150,:,:,:]
# volume = np.flip(small, axis=0)

np.save(DataPath, volume)


In [14]:
volume = np.flip(small, axis=0)
np.save(DataPath, volume)
